# Geological Interpretor Development

This is a notebook for testing and developping some of the basic code in this package.

## Testing ontology manipulation

The knowledge manipulated in this package is formalised in an ontology,<br>
which is store in a *.owl* file.

It is named **MOGI** for **M**inimal **O**ntology for **G**eological **I**nterpretation

To manipulated this ontology, we use the package **owlready2** available from here: https://owlready2.readthedocs.io

In [ ]:
import owlready2 as owl

In [ ]:
owl.onto_path.append("../ontologies/")
mogi = owl.get_ontology("mogi.owl").load()
mogi

Ontology provides access to its components, e.g.:
* classes
* properties
* individuals
* rules

In [ ]:
print(list(mogi.classes()))
print(list(mogi.properties()))
print(list(mogi.individuals()))
print(list(mogi.rules()))

More specific elements can be searched through simple queries:

In [ ]:
mogi.search(iri = "*Surface")

### Reasoner

Ontologies are even more powerful thansk to their capabilities to use reasoning for infering types, properties, and relationships that were not explicitly stated.
This is usefull for obtaining results implied by the already stated information.

This is achieved by running a *reasoner* on the ontology as follows.

In [ ]:
owl.sync_reasoner(infer_property_values=True)

## Creating a dataset

Data are actually described within the ontology, here thanks to the *Data* class.<br>
Adding new data points calls for creating new *Data* individuals (i.e., instances in the ontology).

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data_head = np.array(['name', 'x', 'y', 'z', 'dip_dir', 'dip', 'geology'])
data_array = np.array([['D1', 15, 20, 35, 270, 45, 'Trias_Base'],
                       ['D2', 30, 25, 50, 270, 45, 'Trias_Base'],
                       ['D3', 60, 30, 40, 90, 45, 'Trias_Base'],
                       ['D4', 75, 15, 25, 90, 45, 'Trias_Base'],
                       ['D5', 110, 20, 40, 270, 63, 'Trias_Base'],
                       ['D6', 120, 20, 60, 270, 64, 'Trias_Base'],
                       ['D7', 155, 20, 60, 89, 39, 'Trias_Base'],
                       ['D8', 190, 20, 30, 91, 40, 'Trias_Base'],
                       ['D11', 25, 22, 45, np.nan, np.nan, np.nan],
                       ['D22', 50, 22, 50, np.nan, np.nan, np.nan],
                       ['D44', 100, 30, 20, np.nan, np.nan, np.nan],
                       ['D77', 168, 30, 47, np.nan, np.nan, np.nan]]
)
dataset = pd.DataFrame(data = data_array, columns = data_head)
dataset.set_index("name", inplace = True)
dataset

In [ ]:
# clearing any data already stored in the project
for data_i in mogi.search(type = mogi.Ponctual_Observation):
    owl.destroy_entity(data_i)
mogi.search(type = mogi.Ponctual_Observation)

In [ ]:
# setting the dataset in the ontology by creating individuals
for name_i, values_i in dataset.iterrows():
    mogi.Ponctual_Observation(name_i, **{key:[val] for key, val in values_i.items()})
mogi.search(type = mogi.Ponctual_Observation)

In [ ]:
# for loading dataset from the ontology
dataset = pd.DataFrame(columns=["name","x","y","z","dip_dir","dip",'geology'])
dataset.set_index("name")
for di in mogi.search(type = mogi.Ponctual_Observation):
    for prop in di.get_properties():
        for value in prop[di]:
            dataset.loc[di.name,prop.name] = value
dataset.drop(columns="name", inplace=True)
dataset.reset_index(inplace=True,names="name")
dataset.head()